**This is my personal notebook, I'm just trying out. But feel free to educate me! Any tip would be appreciated.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#0. Get the data
import pandas as pd
data = pd.read_csv("/kaggle/input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-09-14.csv")
data.tail()

In [ ]:
#1. Prepare the data
#Take a look at missing data
import numpy as np
missing_values = data.isnull().sum() #how many values are missing
total_cells = np.prod(data.shape)
total_missings = missing_values.sum()

percent_missings = (total_missings / total_cells) * 100
percent_missings

In [ ]:
#Drop missing values
data = data.dropna()
#Or fill missing values with data.fillna(method='bfill', axis=0).fillna(0) in this case with the value before.

In [ ]:
#Create the input data
X = data.iloc[:, 0:7] #input data
X.tail()

In [ ]:
#Create the target data
y = data.iloc[:, 7]
y.tail()

In [ ]:
#preprocess the input data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

#1 day = 1,440 minutes
#30 days = 43,200 minutes
#60 days = 86,400 minutes
#90 days = 129,600 minutes
X_scaled = X_scaled[len(X_scaled) - 129600:] #data from the last 90 days
y = y[len(y) - 129600:].values #convert from Series to ndarray.
print("X_scaled\'s shape: {}".format(X_scaled.shape))
print("y\'s shape: {}".format(y.shape))

In [ ]:
type(y)

In [ ]:
#create the train, test, and validation set
from sklearn.model_selection import train_test_split
X_train_full,X_test, y_train_full,y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=1)
X_train,X_valid, y_train,y_valid = train_test_split(X_train_full, y_train_full, test_size=0.15, random_state=1)

Regression MLPs (predict a single output given many features, one output neuron)

(for more output = more output neurons a neuron per output)

(No activation fn for the output neurons, so they are free to output any range of values)

(loss fn is typically mse if lot of outliers, or else mae, or Huber loss (combination of mse and mae)) 

In [ ]:
#Create a simple NN model
import tensorflow as tf
from tensorflow import keras
from numpy.random import seed
seed(1)
model = keras.models.Sequential([
    keras.layers.Dense(300, activation='relu', input_shape=(7,)),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(1, activation='relu')
])

optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='mean_squared_logarithmic_error', optimizer=optimizer, metrics=['mse'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

In [ ]:
model.evaluate(X_test, y_test)[1] #(loss, metrics)